In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import certifi
import urllib3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from urllib.parse import urljoin

import re


In [21]:
posibles_subcadenas = [
    # Generales en e-commerce
    '/product/', '/products/', '/producto/', '/productos/',
    '/item/', '/items/', '/detalle/', '/detail/',
    '/sku/', '/articulo/', '/artículos/',

    # Términos comunes en español e inglés
    '/detalle-producto/', '/ver-producto/', '/ver_producto/',
    '/viewproduct/', '/product-detail/', '/product_info/',

    # Prefijos o patrones típicos
    '-p-', '-prod-', '-item-', '-sku-', '-detalle-',

    # Patrones Amazon
    '/dp/', '/gp/product/',

    # Patrones MercadoLibre (por país)
    '/mla-', '/mlm-', '/mlc-', '/mlv-', '/mlu-', '/mlb-', '/mls-',  # Argentina, México, Chile, Venezuela, Uruguay, Brasil, Colombia

    # Easy y Sodimac (comunes en LATAM)
    '/producto/', '/productos/', '/sku/', '/ficha/', '/ficha-producto/',

    # Stanley, DeWalt, Bosch, Makita, etc.
    '/tools/', '/catalog/', '/categories/', '/details/', '/item-details/',

    # Otros posibles patrones semánticos
    '/shop/', '/buy/', '/comprar/', '/oferta/', '/ofertas/', '/promo/', '/promocion/',
    
    # Extensiones finales sospechosas
    '.html', '.htm'
  ]


In [2]:
ruta=r'/home/sebastian/Documentos/programas/Webscrapping/all_links.xlsx'
df_urls=pd.read_excel(ruta,header=0,sheet_name='urls')
df_urls['url final']=df_urls['url final'].str.replace(" ", "", regex=False)
df_urls['url final'] = df_urls['url final'].str.strip()
df_urls.head()

,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion,url final
0,ARG,Argentina,abc sa,SBD & OTHER,scrolldown,mi,no pag,https://www.abc-sa.com.ar/catalogue/all
1,ARG,Argentina,Black & Decker,SBD,page,sec+1,1,https://ar.blackanddecker.global/productos/her...
2,ARG,Argentina,Black & Decker,SBD,page,sec+1,1,https://ar.blackanddecker.global/productos/her...
3,ARG,Argentina,cetrogar,SBD & OTHER,page,sec+1,1,https://www.cetrogar.com.ar/herramientas.html?...
4,ARG,Argentina,Dewalt,SBD,page,sec+1,1,https://ar.dewalt.global/productos/accesorios?...


In [3]:

def validar_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
        }
        #Se obtiene el html de la url
        respuesta = requests.get(url, headers=headers, verify=False, timeout=60)

        if respuesta.status_code == 200:
            #Se convierte el html a un objeto navegable con Beautiful
            soup = BeautifulSoup(respuesta.text, 'html.parser')

            # Buscamos un botón o link de "siguiente"
            paginacion = soup.find_all(lambda tag: (
                tag.name in ['a', 'button'] and
                any(x in tag.get_text(strip=True).lower() for x in ['siguiente', 'sig', '›', '»', '→', 'adelante', 'próximo',
                                                                    'next', '>', '>>', '→', 'forward','próximo', 'seguinte','>', '>>', '›', '»', '→', '→'])
            ))

            if paginacion:
                print(f"[✓] Paginación detectada en {url}")
                return 0, respuesta
            else:
                print(f"[!] Página cargó pero sin controles de paginación visibles: {url}")
                return 2, respuesta  # 2 indica que cargó pero sin paginación

        else:
            print(f"[✗] Error al acceder a {url}: {respuesta.status_code}")
            return 1, None

    except requests.exceptions.RequestException as e:
        print(f"[!] Error de red al acceder a {url}: {e}")
        return 1, None

In [4]:

def obtener_links_web_paginada(respuesta, url,posibles_subcadenas):
    #Se convierte el html a un objeto navegable con Beautiful
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    links = set()


    #busca todos los elemtos'a' que poseen href
    for a in soup.find_all('a', href=True):
        #extrae el hipervinculo de a
        href = a['href']
        href_limpio = href.lower()

        es_producto = (
            any(sub in href_limpio for sub in posibles_subcadenas)
            or any(re.search(r'product|item|skcard|detail|category|categories|filter|sort|search|#', clase, re.I) for clase in a.get('class', []))
            or any(attr in a.attrs for attr in ['data-product-id', 'data-sku', 'data-id', 'data-item-id'])
        )

        # Ignorar enlaces que son anclas, filtros o categorías
        if es_producto and not re.search(r'category|categories|filter|#', href_limpio):
            full_url = urljoin(url, href)
            links.add(full_url)

    return list(links)


In [5]:

# Función para procesar cada URL base y hacer scraping de las páginas
def extraer_links_url(url_base, sec_pag):
    ingreso = 0
    number_page = 0
    links_url = []


    while ingreso < 2:
        # Reemplazo de num_pag
        if isinstance(url_base, str) and 'num_pag' in url_base:
            url = url_base.replace('num_pag', str(number_page))
        else:
            url = url_base

        ingreso, respuesta = validar_url(url)

        if ingreso == 0:
            links_pagina = obtener_links_web_paginada(respuesta, url)
            print(f'Extraer links: se obtuvieron {len(links_pagina)} links de :{url}')

            # 👇 Si no hay links, asumimos que no hay más productos
            if len(links_pagina)==0:
                print(f"Extraer links: [🛑] Página {number_page} sin contenido, deteniendo scraping.")
                ingreso+=1
                break

            links_url.extend(links_pagina)
            number_page += sec_pag
            ingreso = 0
        else:
            ingreso += 1

        time.sleep(10)  # Pausa para respetar el servidor

    return links_url

In [6]:
df_urls.columns

Index(['Code Country', 'Country', 'Name', 'Information', 'Type Pagination',
       'Note', 'Secuencia de paginacion', 'url final'],
      dtype='object')

In [7]:
def construir_dataframe_links(df_urls,url_base,lista_links):    
    # Obtener datos de contexto desde df_urls usando url_base como índice
    Code_Country         = df_urls[df_urls['url final'] == url_base]['Code Country'].values[0]
    Country              = df_urls[df_urls['url final'] == url_base]['Country'].values[0]
    Name                 = df_urls[df_urls['url final'] == url_base]['Name'].values[0]
    Information          = df_urls[df_urls['url final'] == url_base]['Information'].values[0]
    Type_Pagination      = df_urls[df_urls['url final'] == url_base]['Type Pagination'].values[0]
    Note                 = df_urls[df_urls['url final'] == url_base]['Note'].values[0]
    Secuencia_paginacion = df_urls[df_urls['url final'] == url_base]['Secuencia de paginacion'].values[0]


    # Crear DataFrame con los datos
    df_resultado = pd.DataFrame({
        'url final': lista_links,
        'Code Country': Code_Country,
        'Country': Country,
        'Name': Name,
        'Information': Information,
        'Type Pagination': Type_Pagination,
        'Note': Note,
        'Secuencia de paginacion': Secuencia_paginacion
    })
    print(f'contruir dataframe links: se creo el dataframe con los links de {lista_links}')
    return df_resultado

In [8]:
def Crear_base_links(df_urls):
    df_links_webpaginadas = pd.DataFrame()
    for url_base in df_urls['url final']:
        print(url_base)
        Secuencia_paginacion = int(df_urls[df_urls['url final'] == url_base]['Secuencia de paginacion'].values[0])
        print(Secuencia_paginacion)
        lista_links = extraer_links_url(url_base, Secuencia_paginacion)
        print("extraer_links_url")
        df_links_por_url_base = construir_dataframe_links(df_urls,url_base,lista_links)
        print("contruir_dataframe_links")
        df_links_webpaginadas = pd.concat(
            [df_links_webpaginadas, df_links_por_url_base],
            ignore_index=True
        )
        
    return df_links_webpaginadas
        


In [9]:
df_prueba=df_urls.iloc[[4]]
df_prueba

,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion,url final
4,ARG,Argentina,Dewalt,SBD,page,sec+1,1,https://ar.dewalt.global/productos/accesorios?...


In [10]:
df_prueba=df_urls.iloc[[4]]

df_links_webpaginadas=Crear_base_links(df_prueba)

https://ar.dewalt.global/productos/accesorios?page=num_pag
1
[✓] Paginación detectada en https://ar.dewalt.global/productos/accesorios?page=0
Extraer links: se obtuvieron 63 links de :https://ar.dewalt.global/productos/accesorios?page=0
[✓] Paginación detectada en https://ar.dewalt.global/productos/accesorios?page=1
Extraer links: se obtuvieron 63 links de :https://ar.dewalt.global/productos/accesorios?page=1
[✓] Paginación detectada en https://ar.dewalt.global/productos/accesorios?page=2
Extraer links: se obtuvieron 63 links de :https://ar.dewalt.global/productos/accesorios?page=2
[✓] Paginación detectada en https://ar.dewalt.global/productos/accesorios?page=3
Extraer links: se obtuvieron 63 links de :https://ar.dewalt.global/productos/accesorios?page=3
[✓] Paginación detectada en https://ar.dewalt.global/productos/accesorios?page=4
Extraer links: se obtuvieron 63 links de :https://ar.dewalt.global/productos/accesorios?page=4
[✓] Paginación detectada en https://ar.dewalt.global/produc

In [11]:
ruta=r'/home/sebastian/Documentos/programas/Webscrapping/prueba.xlsx'
df_links_webpaginadas.to_excel(ruta)

In [ ]:
df_links_webpaginadas

In [20]:
x='https://ar.dewalt.global/producto/dw2054/guia-magnetica-para-destornillador?tid=572526'
df_links_webpaginadas[df_links_webpaginadas['url final']==x]

,url final,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion
680,https://ar.dewalt.global/producto/dw2054/guia-...,ARG,Argentina,Dewalt,SBD,page,sec+1,1


In [27]:

def validar_urls_paralelo(urls):
    resultados = []

    with ThreadPoolExecutor(max_workers=50) as executor:
        future_to_url = {executor.submit(validar_url, url): url for url in urls}

        # tqdm sobre el iterable de futures que van completándose
        for future in tqdm(as_completed(future_to_url), total=len(urls), desc="Validando URLs"):
            url = future_to_url[future]
            try:
                resultado = future.result()
            except Exception as exc:
                resultado = f'Error interno: {exc}'
            resultados.append((url, resultado))

    return pd.DataFrame(resultados, columns=['url final', 'validacion_url'])

In [ ]:
df_resultado = validar_urls_paralelo(links['url final'])

In [29]:
ruta=r'/home/sebastian/Documentos/programas/Webscrapping/validacion.xlsx'
df_resultado.to_excel(ruta)

# Web Scrapping

## Scrapping Web Paginada

In [ ]:

# Llamar a la función para cada URL base
for url_base in urls_base4:
    extraer_links_webPaginada(url_base)

# Crear un DataFrame con los enlaces de productos
df_link_skuP1 = pd.DataFrame(enlaces_productos, columns=['link_producto'])
# Ajustar la configuración de pandas para mostrar cadenas completas
pd.set_option('display.max_colwidth', None)

# Mostrar el DataFrame
print(df_link_skuP1.head(-5))
print("---------------------------------------------------------------------")
print(f'Se obtuvieron un total de:{len(df_link_skuP1)} links de productos')
print("---------------------------------------------------------------------")

ruta = r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Dashboards LAG\Proyects\WebScrapping\sku_links2.xlsx'
df_link_skuP1.to_excel(ruta, index=False)